In [8]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from torchvision import transforms,datasets
from torch import nn,optim
from torchvision.datasets import MNIST

import matplotlib.pyplot as plt
print(torch.cuda.is_available())  # 应该返回True
print(torch.cuda.device_count())  # 返回可用的GPU数量

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')


True
1
Using device: cuda


In [9]:
# 载入训练集数据，同时把数据转换为tensor，同时下载数据
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)

# 载入测试集数据，同时把数据转换为tensor，同时下载数据
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)

# 文件分别为训练集测试集他们分别的数据和标签

AttributeError: 'MNIST' object has no attribute 'to'

In [3]:
# 批次大小
batch_size = 64
# 装载数据集,dataloader为数据的装载器，数据来源于dataset=train_dataset, 大小为batch_size=batch_size，方式为随机打乱
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

# 展示数据
for i,data in enumerate(train_loader):
    inputs, labels = data
    print(inputs.shape)
    print(labels.shape)
    break

torch.Size([64, 1, 28, 28])
torch.Size([64])


In [4]:
class Net(nn.Module):
    # 初始化，定义网络结构
    def __init__(self):
        # 初始化nnModule
        super(Net, self).__init__()
        # 定义了一个全连接层，参数为输入值和输出值个数
        # 由于是非线性回归，一层神经网络无法做到，因此需要加入隐藏层
        self.fc1 = nn.Linear(784, 10)
        # 设置激活函数
        self.softmax = nn.Softmax(dim=1) # 对第一个维度计算概率值
        
    # 前向计算，定义网络计算
    def forward(self, x):
        # ([64, 1, 28, 28])->([64,784])
        # 由于第0维度取64，所以-1的位置取28*28
        x = x.view(x.size()[0], -1)
        x = self.fc1(x)
        outs = self.softmax(x)
        return outs

In [5]:
# 定义模型
model = Net()

# 定义损失函数
mse_loss = nn.MSELoss()
# 优化器
optimizer = optim.Adam(model.parameters(), lr=0.005)

In [6]:
def train():
    for i,data in enumerate(train_loader):
        # 获得一个批次的数据和标签
        inputs, labels = data
        out = model(inputs)
        # （64）to（64,1）
        labels = labels.reshape(-1,1)
        # to onehot
        # tensor.scatter(dim, index, src)
        # dim:对哪个维度进行独热编码
        # index:要将src中对应的值放到tensor的哪个位置
        # src:插入index的数值
        one_hot = torch.zeros(inputs.shape[0],10).scatter(1,labels,1)
        # 计算loss
        loss = mse_loss(out, one_hot)
        # 梯度清零，计算梯度，修改权值
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def test():
    correct = 0
    for i, data in enumerate(test_loader):
        inputs, labels = data
        out = model(inputs)
        # 计算out中，最大值所在位置
        _, predicted = torch.max(out.data, 1)
        
        correct += (predicted == labels).sum()
    print(' ',correct.item())
    print("Test acc:{0}".format(correct.item() / len(test_loader.dataset)))
        

In [7]:
for epoch in range(10):
    
    train()
    print('epoch:', epoch,' train over')
    
    test()
    
    
# 一共1w张测试集

epoch: 0  train over
  9218
Test acc:0.9218
epoch: 1  train over
  9268
Test acc:0.9268
epoch: 2  train over
  9286
Test acc:0.9286
epoch: 3  train over
  9202
Test acc:0.9202
epoch: 4  train over
  9273
Test acc:0.9273
epoch: 5  train over
  9322
Test acc:0.9322
epoch: 6  train over
  9284
Test acc:0.9284
epoch: 7  train over
  9289
Test acc:0.9289
epoch: 8  train over
  9303
Test acc:0.9303
epoch: 9  train over
  9284
Test acc:0.9284
